Steps:
1. Make blocks from GFF
2. Assign each SNP to a block
3. Use indices of GT to make a DataFrame per block, and remove missing values
4. Use sampling state to sample two haplotypes from each block DF
5. Count number of segregating sites between blocks

In [1]:
import pandas as pd
import numpy as np
import random
import allel
import zarr
import itertools

## (1) Make blocks from GFF

In [2]:
gff = pd.read_table("/Users/s2341012/Dropbox/DISMaL_chapter/brenthis_test/brenthis_data/brenthis_ino.SP_BI_364.v2_0.sequences.braker.gt.gff3",
               nrows=1000,
               usecols=[0, 2, 3, 4],
               sep="\t",
               comment="#",
               header=None)

gff.columns = ["chr", "feature", "start", "end"]

def subset_introns(gff):
    return gff[gff.iloc[:,1] == "intron"]

def subset_length(gff, length):
    return gff[gff.iloc[:,3] - gff.iloc[:,2] > length]

def filter_ld(gff, ld_bp, sort=True):
    #if sort:
     #   gff = gff.sort_values(by=["start", "end"])
    
    filtered = np.where([gff.iloc[row, -2] - gff.iloc[row-1, -1] > ld_bp for row in range(1, len(gff)-1)])

    return gff.iloc[filtered[0], :]

def trim_blocks(gff, length):
    start_indices = [random.randint(gff.iloc[row, -2], (gff.iloc[row, -1]-length)) for row in range(0, len(gff))]
    end_indices = np.array(start_indices) + length+1
    return pd.DataFrame({"chr":gff.iloc[:,0], "start":start_indices, "end":end_indices})

def make_block_indices(gff, blocklength, ld_bp, nrows=None):

    gff = pd.read_table(gff,
               nrows=nrows,
               usecols=[0, 2, 3, 4],
               sep="\t",
               comment="#",
               header=None)

    gff.columns = ["chr", "feature", "start", "end"]

    introns = subset_introns(gff)
    introns_filtered_len = subset_length(introns, blocklength)
    introns_filtered_ld = filter_ld(introns_filtered_len, ld_bp)
    block_indices = trim_blocks(introns_filtered_ld, blocklength)
    block_indices["block_id"] = block_indices["chr"] + ":" + block_indices["start"].astype(str) + "-" + block_indices["end"].astype(str)

    return block_indices

blocks = make_block_indices("/Users/s2341012/Dropbox/DISMaL_chapter/brenthis_test/brenthis_data/brenthis_ino.SP_BI_364.v2_0.sequences.braker.gt.gff3", 100, 10000)
blocks

,chr,start,end,block_id
35,brenthis_ino.SP_BI_364.chromosome_1,73756,73857,brenthis_ino.SP_BI_364.chromosome_1:73756-73857
50,brenthis_ino.SP_BI_364.chromosome_1,136255,136356,brenthis_ino.SP_BI_364.chromosome_1:136255-136356
127,brenthis_ino.SP_BI_364.chromosome_1,237704,237805,brenthis_ino.SP_BI_364.chromosome_1:237704-237805
187,brenthis_ino.SP_BI_364.chromosome_1,280192,280293,brenthis_ino.SP_BI_364.chromosome_1:280192-280293
359,brenthis_ino.SP_BI_364.chromosome_1,352276,352377,brenthis_ino.SP_BI_364.chromosome_1:352276-352377
...,...,...,...,...
483342,brenthis_ino.SP_BI_364.scaffold_2,1661,1762,brenthis_ino.SP_BI_364.scaffold_2:1661-1762
483447,brenthis_ino.SP_BI_364.scaffold_5,11666,11767,brenthis_ino.SP_BI_364.scaffold_5:11666-11767
483477,brenthis_ino.SP_BI_364.scaffold_5,45471,45572,brenthis_ino.SP_BI_364.scaffold_5:45471-45572
483621,brenthis_ino.SP_BI_364.scaffold_8,40518,40619,brenthis_ino.SP_BI_364.scaffold_8:40518-40619


## (2) Assign each SNP to a block

In [3]:
vcf_path = "/Users/s2341012/Dropbox/DISMaL_chapter/brenthis_test/brenthis_data/brenthis_ino_daphne.vcf.gz"
zarr_path = "/Users/s2341012/Dropbox/DISMaL_chapter/brenthis_test/brenthis_data/zarrstore"

try:
    allel.vcf_to_zarr(vcf_path, zarr_path,
                          fields=["samples", "calldata/GT", "variants/CHROM", "variants/POS"], overwrite=False)
except Exception: # if store already exists, proceed
    pass 

callset = zarr.open_group(zarr_path, mode='r')
chromosomes = callset["variants/CHROM"][:]
samples = callset["samples"][:]
gt = callset["calldata/GT"][:]
pos = callset["variants/POS"][:]

callset_positions = pd.DataFrame({"chr":chromosomes, "pos":pos})

/Users/s2341012/mambaforge/envs/dismal/lib/python3.11/site-packages/allel/io/vcf_read.py:1721: UserWarning: invalid FILTER header: '##FILTER=<ID=NonSnp,Description="Set if true: TYPE!=\\"snp\\"">\n'
  warnings.warn('invalid FILTER header: %r' % header)


In [4]:
def get_snp_positions(chromosomes, pos, blocks):
    """Get DF [chr, pos, block, gt_idx]"""
    positions = pd.DataFrame({"chr":chromosomes, "pos":pos})
    positions["gt_idx"] = positions.index

    blocks_snps_df = pd.concat([positions[positions["pos"].between(blocks.iloc[row, 1],
                                                                blocks.iloc[row, 2])]
                                                                  for row in range(len(blocks))])
  
    snp_pos_block = []

    blocks_snps_df_ = blocks_snps_df.reset_index()

    for row in range(len(blocks_snps_df_)):
      position = blocks_snps_df_.iloc[row, 2]
      snp_block = blocks[(blocks["start"].astype(int) <= position) 
                         & (blocks["end"].astype(int) >= position)][["chr", "block_id"]]
      snp_pos_block.append((snp_block.iloc[0][0], position, snp_block.iloc[0][1]))

    snp_blocks = pd.DataFrame(snp_pos_block)
    snp_blocks.columns = ["chr", "pos", "block_id"]

    snp_blocks = snp_blocks.merge(positions, on=["chr", "pos"])
      
    return snp_blocks

get_snp_positions(chromosomes, pos, blocks)

,chr,pos,block_id,gt_idx
0,brenthis_ino.SP_BI_364.chromosome_1,73762,brenthis_ino.SP_BI_364.chromosome_1:73756-73857,1596
1,brenthis_ino.SP_BI_364.chromosome_1,73762,brenthis_ino.SP_BI_364.chromosome_1:73756-73857,1596
2,brenthis_ino.SP_BI_364.chromosome_1,73824,brenthis_ino.SP_BI_364.chromosome_1:73756-73857,1597
3,brenthis_ino.SP_BI_364.chromosome_1,73824,brenthis_ino.SP_BI_364.chromosome_1:73756-73857,1597
4,brenthis_ino.SP_BI_364.chromosome_1,73825,brenthis_ino.SP_BI_364.chromosome_1:73756-73857,1598
...,...,...,...,...
40765,brenthis_ino.SP_BI_364.scaffold_8,40617,brenthis_ino.SP_BI_364.scaffold_8:40518-40619,22239363
40766,brenthis_ino.SP_BI_364.scaffold_8,40617,brenthis_ino.SP_BI_364.scaffold_8:40518-40619,22239363
40767,brenthis_ino.SP_BI_364.scaffold_8,40584,brenthis_ino.SP_BI_364.scaffold_8:40518-40619,22239359
40768,brenthis_ino.SP_BI_364.scaffold_8,40586,brenthis_ino.SP_BI_364.scaffold_8:40518-40619,22239360


## (3) Use GT information to make a DF of calls per block

In [5]:
def block_callset_df(gt, gt_idxs, samples_list, ploidy):
    block_calls = pd.DataFrame(list(itertools.product(samples_list, list(range(ploidy)))),
                        columns=["sample", "hap"])
    
    for idx in gt_idxs:
        block_calls[idx] = gt[idx].flatten()

    return block_calls.dropna()

block_callset_df(gt, list(range(1595, 1598)), samples, 2)

,sample,hap,1595,1596,1597
0,SE_BI_1495,0,0,0,0
1,SE_BI_1495,1,0,0,0
2,FR_BI_1497,0,0,0,0
3,FR_BI_1497,1,0,0,1
4,FR_BD_1329,0,0,1,1
5,FR_BD_1329,1,0,1,1
6,RS_BI_1496,0,0,0,0
7,RS_BI_1496,1,1,0,1
8,RO_BD_956,0,0,1,1
9,RO_BD_956,1,0,1,1


In [6]:
def n_segr_sites(df):
    # count segregating sites between samples of length 2
    return sum(df.iloc[0, 2:] != df.iloc[1, 2:])

In [11]:
def segregating_sites_spectrum(callset, snp_positions, ploidy, sampling_probabilities, samples_df, blocklen):

    rng = np.random.default_rng()

    populations = samples_df["population"].unique()
    pop1 = list(samples_df["sample"][samples_df["population"] == populations[0]])
    pop2 = list(samples_df["sample"][samples_df["population"] == populations[1]])

    sss = np.zeros(shape=(3, blocklen))

    for block in snp_positions["block_id"].unique():
        block_gt_idxs = list(snp_positions[snp_positions["block_id"] == block]["gt_idx"])
        block_callset = block_callset_df(callset["calldata/GT"],
                                          block_gt_idxs,
                                            callset["samples"],
                                              ploidy=ploidy)

        sampling_state = int(np.where(rng.multinomial(n=1, pvals=sampling_probabilities) == 1)[0]) + 1

        if sampling_state == 1:
            block_sss = n_segr_sites(block_callset[block_callset["sample"].isin(pop1)].sample(n=2))
        elif sampling_state == 2:
            block_sss = n_segr_sites(block_callset[block_callset["sample"].isin(pop1)].sample(n=2))
        else:
            assert sampling_state == 3
            block_sss = n_segr_sites(pd.concat([block_callset[block_callset["sample"].isin(pop1)].sample(n=1),
                                     block_callset[block_callset["sample"].isin(pop1)].sample(n=2)]))

        sss[sampling_state-1, int(block_sss)] = sss[sampling_state-1, int(block_sss)] + 1

    return sss

In [8]:
list(callset["samples"])

['SE_BI_1495',
 'FR_BI_1497',
 'FR_BD_1329',
 'RS_BI_1496',
 'RO_BD_956',
 'ES_BI_375',
 'UA_BI_1494',
 'ES_BD_1141',
 'ES_BI_364',
 'ES_BD_1489',
 'ES_BD_1490',
 'GR_BD_1491',
 'IT_BD_1493']

In [9]:
samples_df = pd.DataFrame(
    {
        "sample": callset["samples"][:],
        "population": ["BI", "BI", "BD", "BI", "BD", "BI", "BI", "BD", "BI", "BD", "BD", "BD", "BD"]
    }
)

In [12]:
s3 = segregating_sites_spectrum(
    callset,
    snp_positions=get_snp_positions(callset["variants/CHROM"][:], pos=callset["variants/POS"][:], blocks=blocks),
    ploidy=2,
    sampling_probabilities=[0.25, 0.25, 0.5],
    samples_df=samples_df,
    blocklen=100
)

In [13]:
s3

array([[444., 263., 130., 104.,  57.,  24.,  23.,  11.,  12.,   2.,   3.,
          1.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [429., 266., 138.,  77.,  46.,  20.,  19.,  15.,   9.,   1.,   2.,
          0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0